# &nbsp; 0. Init Colab

In [1]:
%%capture
!pip install geopandas==0.10.2
!pip install mapclassify==2.4.3
!pip install selenium==4.2.0
!pip install pygeos==0.10.2
!pip install -U folium
!apt-get install libspatialindex-dev
!pip install rtree==0.9.7
!pip install -U osmnx==1.1.1
!pip install --upgrade numpy
# !pip install pandana==0.6.1
# !pip install pyrosm==0.6.1

In [2]:
from folium import GeoJson, GeoJsonTooltip, Map, Marker, Icon, PolyLine, FeatureGroup
from shapely.geometry import LineString
from geopy.geocoders import Nominatim
from folium.map import LayerControl
import matplotlib.pyplot as plt
from google.colab import drive
import plotly.express as px
from tqdm.auto import tqdm
import geopandas as gpd
import seaborn as sns
import pandas as pd
import osmnx  as ox
import numpy as np
import sys, os, re
import mapclassify
import datetime
# import pandana 
import folium
import pprint
# import pyrosm
import rtree
import json

drive.mount('/content/drive')

folder_directory = "/content/drive/MyDrive/_____SHARED/Geospatial"
pprint = pprint.PrettyPrinter(indent=4).pprint
sys.path.append(folder_directory)
os.chdir(folder_directory)

import utils, styles
tiles = 'Stamen Terrain'

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Mounted at /content/drive


# &nbsp; 1. SAT Data

Load the data from https://www.sat.tn.it/sentieri/mappa-sentieri/; this data are available under the license 'Open Data Commons Open Database License (ODbL)' and distributed by "© Società degli Alpinisti Tridentini (SAT)".

Alternatively they were provided (but not updated trough the CatalogueServiceWeb("http://geodati.gov.it/RNDT/csw") under the denomination of 'p_TN_f3547bc8-bf1e-4731-85d2-2084d1f4ba07'.

In this data we have the mountain trail maintaned and handled by SAT.

In [3]:
# load the data and selecting fields
sentieri_SAT = gpd.read_file('data/SAT/sentieri_tratte.shp')
selected_cols = ['numero', 'competenza', 'denominaz', 'difficolta', 'loc_fine', 'quota_iniz', 'quota_fine', 't_andata', 't_ritorno', 'geometry']
sentieri_SAT = sentieri_SAT.loc[:, selected_cols]
sentieri_SAT.to_crs(epsg=4326, inplace=True)

In [4]:
# load dolomities data
dolomiti_df = gpd.read_file('data/geo_dolomiti/dolomiti_geo.shp')
dolomiti_df.loc[2, ["name"]] = "Sistema 3 - Pale di San Martino"

# load italian regions & provinces
dolomiti_regions, dolomiti_provincies, dolomiti_municipalities = utils.load_italian_north_east_area()

# buffering on fly on epsg 32632
dolomiti_df.geometry = dolomiti_df.to_crs(epsg=32632).geometry.buffer(300).to_crs(epsg=4326).values
sentieri_SAT_in_dolomities = sentieri_SAT.geometry.apply(lambda x: utils.element_crosses_or_within_area(x, dolomiti_df, 'name'))
assert dolomiti_df.crs == sentieri_SAT.crs

# selecting only the path that are closeby the dolomities
sentieri_SAT['dolomities'] = sentieri_SAT_in_dolomities
sentieri_SAT_in_dolomities = sentieri_SAT.loc[~sentieri_SAT.dolomities.isna()].reset_index(drop=True)
sentieri_SAT_in_dolomities['DEN_PROV'] = sentieri_SAT_in_dolomities.geometry.apply(lambda x: utils.element_within_area(x, dolomiti_provincies, 'DEN_PROV'))
unique_dolomities_group = sentieri_SAT_in_dolomities.dolomities.unique().tolist()

sentieri_SAT_in_dolomities.head(2)

,numero,competenza,denominaz,difficolta,loc_fine,quota_iniz,quota_fine,t_andata,t_ritorno,geometry,dolomities,DEN_PROV
0,E511,SEZ. SAT PREDAZZO,VIA FERRATA DEI CAMPANILI DEL LATEMAR,EEA-D,"FORCELLA GRANDE - Bivacco ""Mario Rigatti""",2570,2634,01:30,01:30,"LINESTRING (11.56540 46.38048, 11.56547 46.380...",Sistema 7 - Sciliar-Catinaccio & Latemar,Trento
1,E714,SAT O.C.,"VIA FERRATA ""NICO GUSELLA""",EEA-D,FORCELLA DEL PORTON,2451,2420,02:00,02:00,"LINESTRING (11.84688 46.24668, 11.84683 46.246...",Sistema 3 - Pale di San Martino,Trento


In [5]:
# Explore in which provinces are the SAT paths
path_per_dolomities = {k : len(sentieri_SAT_in_dolomities.loc[sentieri_SAT_in_dolomities.DEN_PROV == k]) for k in sentieri_SAT_in_dolomities.DEN_PROV.unique()}
for name, value in sorted(path_per_dolomities.items(), key=lambda item: item[1]):
  print(f'Number of path in {name}: {value}')

print('-'*50)
# Explore in which group of dolomities are the SAT paths
path_per_dolomities = {k : len(sentieri_SAT_in_dolomities.loc[sentieri_SAT_in_dolomities.dolomities == k]) for k in unique_dolomities_group}
for name, value in sorted(path_per_dolomities.items(), key=lambda item: item[1]):
  print(f'Number of path in {name}: {value}')

Number of path in None: 0
Number of path in Trento: 112
--------------------------------------------------
Number of path in Sistema 2 - Marmolada: 7
Number of path in Sistema 7 - Sciliar-Catinaccio & Latemar: 29
Number of path in Sistema 3 - Pale di San Martino: 33
Number of path in Sistema 9 - Dolomiti di Brenta: 62


In [6]:
# filter all the dolomities only for the intersted provinces:
dolomiti_df = dolomiti_df.loc[dolomiti_df['name'].isin(sentieri_SAT_in_dolomities.dolomities.unique())]
dolomiti_geo = GeoJson(data = dolomiti_df, style_function = styles.style_dolomiti, tooltip = GeoJsonTooltip(fields=['name', 'url_info', 'area']))

In [7]:
sat_dolomities_map = utils.get_new_map(title='SAT Trail on the Dolomities', tiles=tiles)

# create relevant feature layers
feature_layers = {group:FeatureGroup(name=f"Group of dolomities: {group}") for group in unique_dolomities_group}
feature_layers['dolomiti_area'] =  FeatureGroup(name='Area Dolomiti')
dolomiti_geo.add_to(feature_layers['dolomiti_area'])

# SAT Paths
for _, row in sentieri_SAT_in_dolomities.iterrows():
  line = PolyLine(
    locations = [list(reversed(i)) for i in list(row.geometry.coords)],
    tooltip = utils.get_info_from_row(row),
    popup = utils.get_info_from_row(row),
    color='darkred', weight=3,
  )

  line.add_to(feature_layers[row.dolomities])

utils.add_map_infos(feature_layers, sat_dolomities_map)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
# load previusoly processed data about huts
osm_rifugi = gpd.read_file('data/alpine_huts/osm_dolomities.shp')

# filter the huts so that we have the same dolomities group
osm_rifugi = osm_rifugi.loc[osm_rifugi.dolomities.isin(unique_dolomities_group)]

In [11]:
# huts and paths together
sat_hut_dolomities_map = utils.get_new_map(title='Path & Huts according to Dolomities group', tiles=tiles)

feature_layers = {group:FeatureGroup(name=f"Group of dolomities: {group}") for group in unique_dolomities_group}
feature_layers['dolomiti_area'] =  FeatureGroup(name='Area Dolomiti')
dolomiti_geo.add_to(feature_layers['dolomiti_area'])

# SAT Paths
for _, row in sentieri_SAT_in_dolomities.iterrows():
  line = PolyLine(
    locations = [list(reversed(i)) for i in list(row.geometry.coords)],
    tooltip = utils.get_info_from_row(row),
    popup = utils.get_info_from_row(row),
    color='darkred', weight=3,
  )

  line.add_to(feature_layers[row.dolomities])

# Huts
for idx, row in osm_rifugi.iterrows():
  marker = Marker(
      location = [i[0] for i in reversed(row.geometry.xy)],
      tooltip = utils.get_info_from_row(row),
      popup = utils.get_info_from_row(row),
      icon = Icon(color = styles.c_colors[unique_dolomities_group.index(row.dolomities)])
  )
  marker.add_to(feature_layers[row.dolomities])

utils.add_map_infos(feature_layers, sat_hut_dolomities_map, collapsed_legend=True)

Output hidden; open in https://colab.research.google.com to view.

# Inspect the paths that bring to the hut

In [12]:
osm_rifugi_buffered = osm_rifugi.copy()
osm_rifugi_buffered.geometry = osm_rifugi.to_crs(epsg=32632).buffer(50).to_crs(epsg=4326)

In [13]:
res = []

# saving only the huts that are reached from a SAT path
for i in range(len(osm_rifugi_buffered)):
  intersection_idx = sentieri_SAT_in_dolomities.geometry.crosses(osm_rifugi_buffered.geometry.values[i])
  idx_trues = np.nonzero(intersection_idx.values)[0]

  if idx_trues.size > 0:
    res.append(idx_trues.tolist())
  else:
    res.append(None)

osm_rifugi_buffered['path'] = res
osm_rifugi_buffered = osm_rifugi_buffered.loc[~osm_rifugi_buffered.path.isna()].reset_index(drop=True)
hut_reachable = osm_rifugi_buffered['name'].unique()

In [14]:
# huts and paths together
sat_hut_path_dolomities_map = utils.get_new_map(title='Path & Huts according to Dolomities group', tiles=tiles)

feature_layers = {group:FeatureGroup(name=f"Hut: {group}") for group in hut_reachable}
feature_layers['dolomiti_area'] =  FeatureGroup(name='Area Dolomiti')
dolomiti_geo.add_to(feature_layers['dolomiti_area'])

# Huts - SAT paths
for idx, row in osm_rifugi_buffered.iterrows():
  marker_group = row['name']
  marker = Marker(
      location = [i[0] for i in reversed(row.geometry.centroid.xy)],
      tooltip = utils.get_info_from_row(row),
      popup = utils.get_info_from_row(row),
      icon = Icon(color = styles.c_colors[unique_dolomities_group.index(row.dolomities)])
  )
  marker.add_to(feature_layers[marker_group])

  for path in row.path:
    row = sentieri_SAT_in_dolomities.iloc[path]
    line = PolyLine(
      locations = [list(reversed(i)) for i in list(row.geometry.coords)],
      tooltip = utils.get_info_from_row(row),
      popup = utils.get_info_from_row(row),
      color='darkred', weight=3,
    )

    line.add_to(feature_layers[marker_group])


utils.add_map_infos(feature_layers, sat_hut_path_dolomities_map, collapsed_legend=True)

# Reach the paths from Trento or Bolzano

In [15]:
%%time 
# Querying all the network street of Trentino-Alto Adige
if 'graph.graphml' in os.listdir('data'):
  G = ox.load_graphml('data/graph.graphml')
else:
  G = ox.graph_from_place('Trentino-Alto Adige', network_type='drive')
  ox.save_graphml(G, 'data/graph.graphml')

CPU times: user 11.3 s, sys: 706 ms, total: 12 s
Wall time: 12.4 s


In [16]:
from shapely.geometry import Point

# Since the path are LineString we want to find the starting point
# and the endpoint but we miss this information.

def extract_first_and_last_point_from_line(line_string, return_point=False):
  first = list([i[0] for i in line_string.xy])
  last = list([i[-1] for i in line_string.xy])
  if return_point:
    return Point(first), Point(last)
  else:
    return first, last

nodes_from_line = [
                   extract_first_and_last_point_from_line(line, return_point=False) 
                   for line in sentieri_SAT_in_dolomities.geometry
                  ]

nodes_from_line_points = [
                   extract_first_and_last_point_from_line(line, return_point=True) 
                   for line in sentieri_SAT_in_dolomities.geometry
                  ]

In [17]:
id_nodes_in_graph = []
nearest_point_into_street = []

for idx, (first, last) in enumerate(nodes_from_line):
  first_nearest_point = ox.nearest_nodes(G, first[0], first[1])
  second_nearest_point = ox.nearest_nodes(G, last[0], last[1])

  # find which is the closest to any street
  point_first = Point(
      G.nodes[first_nearest_point]['x'],
      G.nodes[first_nearest_point]['y'],
    )
  
  point_first_distance = point_first.distance(nodes_from_line_points[idx][0])
  point_first_xy = (G.nodes[first_nearest_point]['x'], G.nodes[first_nearest_point]['y'])

  # last
  point_last = Point(
      G.nodes[second_nearest_point]['x'],
      G.nodes[second_nearest_point]['y'],
    )
  
  point_last_distance = point_last.distance(nodes_from_line_points[idx][1])
  point_last_xy = (G.nodes[second_nearest_point]['x'], G.nodes[second_nearest_point]['y'])

  if point_last_distance <= point_first_distance:
    id_nodes_in_graph.append(second_nearest_point)
    nearest_point_into_street.append(point_last_xy)

  else:
    id_nodes_in_graph.append(first_nearest_point)
    nearest_point_into_street.append(point_first_xy)

sentieri_SAT_in_dolomities['closest_point_to_graph_id'] = id_nodes_in_graph
sentieri_SAT_in_dolomities['closest_point_to_graph'] = nearest_point_into_street

In [18]:
sentieri_SAT_in_dolomities['reachable_huts'] = ""

# add information to the path of the hut that is possible to reach
for idx, row in osm_rifugi_buffered.iterrows():
  name = row['name']
  for path in row.path:
    current = sentieri_SAT_in_dolomities.iloc[path]['reachable_huts']
    sentieri_SAT_in_dolomities.loc[path, ['reachable_huts']] = f"{current}, {name}" if current else f"{name}"

In [19]:
# add to huts the trail by which are accessible
trail_to_huts = []
for paths in osm_rifugi_buffered.path:
  tmp = []
  for path in paths:
    tmp.append(sentieri_SAT_in_dolomities.iloc[path].numero)
  trail_to_huts.append(', '.join(tmp))

osm_rifugi_buffered["accessible_via_path"] = trail_to_huts

In [20]:
sat_dolomities_map = utils.get_new_map(title='Access the SAT path from the street', tiles=tiles)
feature_layers = {group:FeatureGroup(name=f"Group of dolomities: {group}") for group in unique_dolomities_group}
feature_layers['dolomiti_area'] =  FeatureGroup(name='Area Dolomiti')
feature_layers['closest_points'] =  FeatureGroup(name='Closest points')
dolomiti_geo.add_to(feature_layers['dolomiti_area'])

# SAT Paths
for idx, row in sentieri_SAT_in_dolomities.iterrows():
  line_group = row.dolomities
  line = PolyLine(
    locations = [list(reversed(i)) for i in list(row.geometry.coords)],
    tooltip = utils.get_info_from_row(row),
    popup = utils.get_info_from_row(row),
    color='darkred', weight=3,
  )

  line.add_to(feature_layers[line_group])

  closest_point = folium.CircleMarker(
      location= list(reversed(row.closest_point_to_graph)),
      tooltip = utils.get_info_from_row(row),
      popup = utils.get_info_from_row(row),
      radius = 5, color="#3186cc",
      weight=4, fill=True, fill_color="#3186cc"
  )

  closest_point.add_to(feature_layers['closest_points'])

for idx, row in osm_rifugi_buffered.iterrows():
  marker_group = row.dolomities
  marker = Marker(
      location = [i[0] for i in reversed(row.geometry.centroid.xy)],
      tooltip = utils.get_info_from_row(row),
      popup = utils.get_info_from_row(row),
      icon = Icon(color = styles.c_colors[unique_dolomities_group.index(row.dolomities)])
  )
  marker.add_to(feature_layers[marker_group])

utils.add_map_infos(feature_layers, sat_dolomities_map, collapsed_legend=True)

Output hidden; open in https://colab.research.google.com to view.

In [21]:
import networkx

def load_centroid_in_graph_common_city(
    G: networkx.classes.multidigraph.MultiDiGraph,
    cities:list = ['Trento', 'Bolzano']
  ) -> dict:
  "return the nodes in the graph"
  
  res = {}
  Trento = (11.121597, 46.065489)
  Bolzano = (11.346474, 46.492602)

  res['Bolzano'] = ox.nearest_nodes(G, Bolzano[0], Bolzano[1])
  res['Trento'] = ox.nearest_nodes(G, Trento[0], Trento[1])
  return res


G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

cities_centroid = load_centroid_in_graph_common_city(G, dolomiti_municipalities)

In [25]:

def generate_route_to_hut(starting_city, selected_path, cities_centroid):
  detail = sentieri_SAT_in_dolomities.loc[sentieri_SAT_in_dolomities['numero'] == selected_path]

  print(f"""\
  Starting from: {starting_city}
  Selected path: {selected_path}
  Dolomities Group: {detail.dolomities.values[0]}
  {'-'*50}
  Denominazione: {detail.denominaz.values[0]}
  Reachable_huts: {detail.reachable_huts.values[0]}
  Competenza: {detail.competenza.values[0]}
  Difficolta: {detail.difficolta.values[0]}
  Tempo andata: {detail.t_andata.values[0]}
  Tempo ritorno: {detail.t_ritorno.values[0]}
  Quota iniziale: {detail.quota_iniz.values[0]}
  Quota Finale: {detail.quota_fine.values[0]}
  {'-'*50}
  """)

  closest_point_to_graph = detail.closest_point_to_graph_id.values[0]

  route = ox.shortest_path(
      G, 
      closest_point_to_graph, 
      cities_centroid[starting_city], 
      weight='travel_time'
    )
  
  edge_lengths = ox.utils_graph.get_route_edge_attributes(G, route, 'length')
  edge_times = ox.utils_graph.get_route_edge_attributes(G, route, 'travel_time')

  print(f"The route is long {round(sum(edge_lengths)/1000)}km")
  print(f"The route takes {str(datetime.timedelta(seconds=sum(edge_times)))}")
  print('\n'+'-'*50, '\n')

  route_map = ox.plot_route_folium(
      G, 
      route, 
      popup_attribute='name', 
      tiles='OpenStreetMap'
    )

  # add information to the map
  feature_layers = {detail.dolomities.values[0] :FeatureGroup(name=f"Group of dolomities: {detail.dolomities.values[0]}")}
  feature_layers['dolomiti_area'] =  FeatureGroup(name='Area Dolomiti')
  feature_layers['closest_points'] =  FeatureGroup(name='Closest points')

  dolomiti_geo_ = GeoJson(data = dolomiti_df.loc[dolomiti_df['name'] == detail.dolomities.values[0]], style_function = styles.style_dolomiti, tooltip=GeoJsonTooltip(fields=['name', 'url_info', 'area']))
  dolomiti_geo_.add_to(feature_layers['dolomiti_area'])

  # SAT Paths
  for idx, row in sentieri_SAT_in_dolomities.iterrows():
    if row.dolomities == detail.dolomities.values[0]:
      line_group = row.dolomities
      line = PolyLine(
        locations = [list(reversed(i)) for i in list(row.geometry.coords)],
        tooltip = utils.get_info_from_row(row),
        popup = utils.get_info_from_row(row),
        color='darkred', weight=3,
      )

      line.add_to(feature_layers[line_group])

      closest_point = folium.CircleMarker(
          location= list(reversed(row.closest_point_to_graph)),
          tooltip = utils.get_info_from_row(row),
          popup = utils.get_info_from_row(row),
          radius=5, color="green",
          weight=4, fill=True, fill_color="green",
      )

      closest_point.add_to(feature_layers['closest_points'])

  # huts
  for idx, row in osm_rifugi_buffered.iterrows():
    if row.dolomities == detail.dolomities.values[0]:
      marker_group = row.dolomities
      marker = Marker(
          location = [i[0] for i in reversed(row.geometry.centroid.xy)],
          tooltip = utils.get_info_from_row(row),
          popup = utils.get_info_from_row(row),
          icon = Icon(color = styles.c_colors[unique_dolomities_group.index(row.dolomities)])
      )
      marker.add_to(feature_layers[marker_group])

  return utils.add_map_infos(feature_layers, route_map, collapsed_legend=True)
   


In [26]:
#@title Distance and best path

starting_city = "Trento" #@param ["Trento", "Bolzano"]
selected_path = "O322" #@param ["O303", "O304", "O307", "E584", "E584", "E584", "O316", "O323", "O325", "O325", "O348", "E540", "E540", "E542", "E542", "E542", "O317", "O318A", "O318A", "O320", "O321", "O321", "O322", "O325B", "O327", "O332", "O340", "O340", "O341", "O341A", "E594", "O358", "E701A", "E703", "E703", "E710", "O319", "O319", "O328", "E739A", "E516", "E541", "E545", "E546", "E546", "E546", "E548", "E550", "E720", "E707", "E707", "E709", "E549", "E549", "E716", "E580", "E583", "E583", "E585", "E715", "E718"]
generate_route_to_hut(starting_city, selected_path, cities_centroid)

  Starting from: Trento
  Selected path: O322
  Dolomities Group: Sistema 9 - Dolomiti di Brenta
  --------------------------------------------------
  Denominazione: SENTIERO ATTREZZATO DELLE VAL PERSE
  Reachable_huts: Rifugio Croz dell'Altissimo
  Competenza: SEZ. SAT MOLVÉNO, SAT O.C.
  Difficolta: EE
  Tempo andata: 04:00
  Tempo ritorno: 02:50
  Quota iniziale: 1323
  Quota Finale: 2461
  --------------------------------------------------
  
The route is long 42km
The route takes 0:35:55.500000

-------------------------------------------------- 

